### Iris Recognition using Full Eye Image

#### 1. Importing Libraries

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import DataLoader, Subset
from torchvision import transforms as tt
from src.lib.cnn_utils import load_model, identification_test_all_vs_all, verification_all_vs_all, \
    generate_embeddings, identification_test_probe_vs_gallery, verification_probe_vs_gallery
from src.data.CasiaIrisDataset import CasiaIrisDataset
from src.data.datasetUtils import splitDataset, split_dataset_gallery_test
from src.utils.plotting import plot_far_frr_roc
import torch

#### 2. Configuration

In [ ]:
# Dataset Path
datasetPath = "F:\\Dataset\\Casia"
# Model Path
modelPath = ".\\models\\modelFullEyeMargin040.pth"
# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### 3. Loading the Model

In [ ]:
net = load_model(modelPath)
if net is None:
    print("Model not found")
    exit(1)
else:
    # Move the model to the device
    net.to(device)
    # Set the model to evaluation mode
    net.eval()

#### 4. Loading the Dataset

In [ ]:
# Transformations
transform = tt.Compose([
        tt.ToTensor(),  # Convert to tensor
])

# Load the dataset
dataset = CasiaIrisDataset(datasetPath, transform=[transform], centered=True)

# Splitting the Dataset: Train(70%), Validation(10%), Test(20%
train_dataset, val_dataset, test_dataset = splitDataset(dataset, 0.2, 0.1)

# Set the dataset to evaluation mode
dataset.eval()

# Load the test dataset
all_vs_all_dataset = DataLoader(dataset=test_dataset, batch_size=64, num_workers=8, pin_memory=True)

#### 5. All vs All Test

##### 5.1. Generate Embeddings

In [ ]:
# Generate embeddings
embedding_list, labels_list = generate_embeddings(device, net, all_vs_all_dataset)

##### 5.2. Merge the Embeddings into a Matrix

In [ ]:
embedding_array = embedding_list.numpy()

# Compute the cosine similarity matrix and convert it to a dissimilarity matrix
M = - cosine_similarity(embedding_array)

# Round the cosine similarity to 3 decimal places
M = np.round(M, 3)

# Normalize the cosine similarity to [0, 1]
M = (M + 1) / 2

##### 5.3. Identification Test

In [ ]:
# Compute the thresholds, DIR, GRR, FAR, FRR
thresholds, DIR, GRR, FAR, FRR = identification_test_all_vs_all(M, labels_list)

# Plot the FAR, FRR, Rank-1, GRR and EER
plot_far_frr_roc(thresholds, FAR, FRR, GRR, DIR=np.array(DIR), roc=True, titleRoc="ROC Curve - Identification - All vs all", titleEer="FAR, FRR, Rank-1, GRR and EER - Identification - All vs all")


##### 5.3. Verification Test

In [ ]:
# Compute the GAR, FAR, FRR, GRR
GAR, FAR, FRR, GRR = verification_all_vs_all(M, labels_list)

# Plot the FAR, FRR, Rank-1, GRR and EER
plot_far_frr_roc(thresholds, FAR, FRR, GRR, DIR=None, titleEer="FAR, FRR, Rank-1, GRR and EER - Verification - All vs all")

#### 6. Probe vs Gallery Test

##### 6.1. Split the Dataset

In [ ]:
# Split the dataset into gallery and probe
gallery, probe = split_dataset_gallery_test(test_dataset, gallery_ratio=0.6, seed=20)

gallery = Subset(test_dataset, gallery)
probe = Subset(test_dataset, probe)

# Load the gallery and probe dataset
gallery_dataset = DataLoader(dataset=gallery, batch_size=64, shuffle=False, num_workers=8, pin_memory=False)
probe_dataset = DataLoader(dataset=probe, batch_size=64, shuffle=False, num_workers=8, pin_memory=False)

##### 6.2. Generate Embeddings

In [ ]:
# Generate embeddings for gallery and probe
embedding_list_gallery, labels_list_gallery = generate_embeddings(device, net, gallery_dataset)
embedding_list_probe, labels_list_probe = generate_embeddings(device, net, probe_dataset)

##### 6.3. Merge the Embeddings into a Matrix

In [ ]:
embedding_list_gallery = embedding_list_gallery.numpy()
embedding_list_probe = embedding_list_probe.numpy()

# Compute the cosine similarity matrix and convert it to a dissimilarity matrix
M = - cosine_similarity(embedding_list_probe, embedding_list_gallery)
# Normalize the cosine similarity to [0, 1]
M = (np.round(M, 3) + 1) / 2

##### 6.4. Identification Test

In [ ]:
# Compute the thresholds, DIR, GRR, FAR, FRR
thresholds, DIR, GRR, FAR, FRR = identification_test_probe_vs_gallery(M, labels_list_probe, labels_list_gallery)

# Plot the FAR, FRR, Rank-1, GRR and EER
plot_far_frr_roc(thresholds, FAR, FRR, GRR, DIR=np.array(DIR), roc=True, titleRoc="ROC Curve - Identification - Probe vs Gallery", titleEer="FAR, FRR, Rank-1, GRR and EER - Indentification - Probe vs Gallery")

##### 6.5. Verification Test

In [ ]:
# Compute the GAR, FAR, FRR, GRR
GAR, FAR, FRR, GRR = verification_probe_vs_gallery(M, labels_list_probe, labels_list_gallery)

# Plot the FAR, FRR, Rank-1, GRR and EER
plot_far_frr_roc(thresholds, FAR, FRR, GRR, DIR=None, titleEer="FAR, FRR, Rank-1, GRR and EER - Verification - Probe vs Gallery")